# Задание 4

Рассматриваемый технологический узел представлен на рисунке ниже:

![Figure for task](.\fig_task_4.png "Схема узла")

Приборы показывают, что в узел поступает 90. По левой трубе уходит 35, а по правой - 60. 

*Допущения:*
 - физическими свойствами транспортируемой среды пренебрегаем;
 - в узле нет ни потерь, ни источников;
 - процесс установившейся.

---

## Решение

1. Для решения поставленной задачи применим методы DRGED - Data Reconciliation and Gross Error Detection (согласование данных и обнаружение грубых ошибок)

Согласование данных — это оценка переменных технологического процесса на основе информации, содержащейся в измерениях и моделях процесса. Модели процессов, используемые при согласовании данных, обычно представляют собой уравнения сохранения массы и энергии.

Основное применение этих методов заключается в подстройке некорретных измерений с помощью технологических инструментов (в том числе с использованием программных комплексов, таких как *IDAES Toolkit*), а также в обнаружении проблем и отказов в измерительном оборудовании.

2. Оптимальные оценки переменных процесса с помощью DR являются решениями ограниченной целевой функции наименьших квадратов или максимального правдоподобия, где ошибки измерения минимизируются с ограничениями модели процесса.

В условиях задания нет набора данных по измерениям потоков, а также отсутствиют величины погрешностей измерительных приборов. Следовательно, непредставляется возможным определить величину стандартных отклонений потоков (вход1, выход1, выход2) и построить матрицу дисперсий `V` для схемы замещения. Что в свою очередь не позволит определить наличие грубой ошибки в измерениях прибора (*gross error*)

3. Определим истинные значения расходов `x` и приведём уравнение баланса в узле:

`x_0 - x_1 - x_2 = 0`
    
где i = 0, 1, 2 - индексы потоков (указаны на рисунке).

Массив измеренных занесём в переменную `Y`. Отметим, что `y_i = x_i + e_i`, где `e_i` - случайная ошибка измерения.

4. Для создания модели и решения задачи нахождения минимума целевой функции введём ограничения, на основе имеющихся данных.

`x_0 - x_1 - x_2 = 0` => `y_0 - e_0 - (y_1 - e_1) - (y_2 - e_2) = 0` =>

=> `90 - e_0 - 60 + e_1 - 35 + e_2 = 0` => `e_0 - e_1 - e_2 = -5`


In [1]:
import numpy as np
from scipy.optimize import minimize

# Имеющиеся данные
# Y - массив измерений потоков
Y = np.array([90.0, 60.0, 35.0])

# определим модель которую будем использовать в целевой функции
model = lambda e, Y: Y[0] - e[0] + Y[1] - e[1] + Y[2] - e[2]

# Целевая функция для минимизации (регрессия наименьших квадратов)
obj = lambda e, Y: np.sum((Y - model(e, Y))**2)

# Границы для e[0], e[1], e[2]
bnds = [(-10, 10), (-10, 10), (-10, 10)]

# Ограничение: e[0] - e[1] - e[2] + 5 = 0
cons = [{"type": "eq", "fun": lambda e: e[0] - e[1] - e[2] + 5}]

# Первоначальная догадка для x[1], x[2], x[3]:
xinit = np.array([-1.0, 0.0, 1.0])

res = minimize(obj, args=(Y), x0=xinit, bounds=bnds, constraints=cons)

e_0, e_1, e_2 = res.x[0], res.x[1], res.x[2]

print(f"e_0 = {e_0:0.2f}, e_1 = {e_1:0.2f}, e_2 = {e_2:0.2f}")

e_0 = 10.00, e_1 = 7.00, e_2 = 8.00


In [5]:
# истинные значения потоков
x_0, x_1, x_2 = Y[0] - e_0, Y[1] - e_1, Y[2] - e_2

print(f"x_0 = {x_0:0.2f}, x_1 = {x_1:0.2f}, x_2 = {x_2:0.2f}")

# Проверяем условие баланса потоков
print(f"Баланс потоков : {x_0 - x_1 - x_2:0.2f}")

x_0 = 80.00, x_1 = 53.00, x_2 = 27.00
Баланс потоков : 0.00


## Сводная таблица

| Номер потока | Измеренное значение | Истинное значение |
| ----------- | ----------- | ----------- |
| 0 | 90.00 | 80.00 |
| 1 | 60.00 | 53.00 |
| 2 | 35.00 | 27.00 |